In [3]:
# Reference: https://qiita.com/Cyber_Hacnosuke/items/122cec35d299c4d01f10
#さっきの
place_codeA = [14, 44, 81]
place_codeB = [47412, 47662, 47762]
place_name = ["札幌", "東京", "下關"]   

import requests
from bs4 import BeautifulSoup #ダウンロードしてなかったらpipでできるからやってね。
import csv

# URLで年と月ごとの設定ができるので%sで指定した英数字を埋め込めるようにします。
base_url = "http://www.data.jma.go.jp/obd/stats/etrn/view/hourly_s1.php?prec_no=%s&block_no=%s&year=2022&month=2&day=%s&view=p1"

#取ったデータをfloat型に変えるやつ。(データが取れなかったとき気象庁は"/"を埋め込んでいるから0に変える)
def str2float(str):
  try:
    return float(str)
  except:
    return 0.0


if __name__ == "__main__":
  #都市を網羅します
  for place in place_name:
    #最終的にデータを集めるリスト (下に書いてある初期値は一行目。つまり、ヘッダー。)
#     All_list = [['年月日', '日照時間(h)', '太陽總輻射量（MJ/㎡）']]
    All_list = []
    print(place)
    index = place_name.index(place)
    # for文で2007年~2017年までの11回。
    for day in range(1,29):
        #2つの都市コードと年と月を当てはめる。
        r = requests.get(base_url%(place_codeA[index], place_codeB[index], day))
        r.encoding = r.apparent_encoding

        # まずはサイトごとスクレイピング
        soup = BeautifulSoup(r.text)
        # findAllで条件に一致するものをすべて抜き出します。
        # 今回の条件はtrタグでclassがmtxになってるものです。
        rows = soup.findAll('tr',class_='mtx')
        # print(rows[8])

        # 表の最初の1~4行目はカラム情報なのでスライスする。(indexだから初めは0だよ)
        # 【追記】2020/3/11 申し訳ございません。間違えてました。
        rows = rows[8:20]

        # 1日〜最終日までの１行を網羅し、取得します。
        for row in rows:
          # 今度はtrのなかのtdをすべて抜き出します
          data = row.findAll('td')

          #１行の中には様々なデータがあるので全部取り出す。
          # ★ポイント
          rowData = [] #初期化
          rowData.append(str(2022) + "/" + str(2) + "/" + str(day) + " " + str(data[0].string))
          rowData.append(str2float(data[10].string))
          rowData.append(str2float(data[11].string))

          #次の行にデータを追加
          All_list.append(rowData)

    #都市ごとにデータをファイルを新しく生成して書き出す。(csvファイル形式。名前は都市名)
    with open(place + '.csv', 'w', encoding='utf-8') as file:
      writer = csv.writer(file, lineterminator='\n')
      writer.writerows(All_list)

札幌
東京
下關
